<a href="https://colab.research.google.com/github/dkaloni/BloodReport_Analytics/blob/main/ANNclassificationonbloodreport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter
from sklearn import preprocessing,svm,metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from pandas import Series, DataFrame

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import pydot
import graphviz
from IPython.display import SVG
import seaborn as sns
import tensorflow as tf
from tensorflow import keras

In [ ]:
data = pd.read_csv('/content/dataall.csv')

In [ ]:

import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

In [ ]:
#!pip install -q -U keras-tuner

In [ ]:
import kerastuner as kt
data = data.drop('patient_id', axis=1)
data.drop(data[data['parameter_value'] == 920000 ].index , inplace=True)
data.drop(data[data['parameter_value'] == 40250 ].index , inplace=True)


In [ ]:
data.drop(data[data['parameter_value'] == 22000 ].index , inplace=True)


In [ ]:
data.drop(data[data['parameter_value'] == 17000 ].index , inplace=True)


In [ ]:
data.describe()

,Chloride,SGPT,Sodium,T4,WBC,female,male,parameter_value,normal,highproblem,lowproblem
count,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000
mean,0.121405,0.131009,0.126507,0.264019,0.357061,0.115752,0.884248,218.871289,0.864439,0.064379,0.043420
std,0.326605,0.337419,0.332428,0.440820,0.479145,0.319935,0.319935,1164.190355,0.342330,0.245433,0.203805
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-71.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.900000,1.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,9.200000,1.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,100.900000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,16100.000000,1.000000,1.000000,1.000000


In [ ]:
data.parameter_value = data.parameter_value/(16100)

In [ ]:
data.describe()

,Chloride,SGPT,Sodium,T4,WBC,female,male,parameter_value,normal,highproblem,lowproblem
count,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000,19991.000000
mean,0.121405,0.131009,0.126507,0.264019,0.357061,0.115752,0.884248,0.013594,0.864439,0.064379,0.043420
std,0.326605,0.337419,0.332428,0.440820,0.479145,0.319935,0.319935,0.072310,0.342330,0.245433,0.203805
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.004410,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000429,1.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000571,1.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.006267,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
X=data.iloc[:,:-1] ## independent features
y=data.iloc[:,-1]

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [ ]:
tuner.search_space_summary()


Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
units             |288               |?                 
learning_rate     |0.0001            |?                 
tuner/epochs      |2                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |2                 |?                 
tuner/round       |0                 |?                 

Epoch 1/2


ValueError: ignored

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)


In [ ]:
tuner.search(X_train, y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
units             |288               |?                 
learning_rate     |0.0001            |?                 
tuner/epochs      |2                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |2                 |?                 
tuner/round       |0                 |?                 

Epoch 1/2


ValueError: ignored

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_test, label_test, epochs=best_epoch)

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

In [ ]:
data = data.drop('patient_id', axis=1)

KeyError: ignored

In [ ]:
X = data.iloc[:,0:11].values
Y = data.iloc[:,11].values

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y1 = encoder.fit_transform(y)
Y = pd.get_dummies(y1).values

from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,test_size = 0.1,random_state = 0)

In [ ]:
y1  

array([2, 2, 0, ..., 0, 0, 0])

In [ ]:
Y

array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       ...,
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0]], dtype=uint8)

In [ ]:
model = Sequential()
model.add(Dense(11, input_shape=(11,), activation = 'relu'))
model.add(Dense(20)) # New hidden layer with 4 params
model.add(Dense(20)) # New hidden layer with 4 params
model.add(Dense(20)) # New hidden layer with 4 params
model.add(Dense(20)) # New hidden layer with 4 params
model.add(Dense(20)) # New hidden layer with 4 params
model.add(Dense(20)) # New hidden layer with 4 params
model.add(Dense(20)) # New hidden layer with 4 params
model.add(Dense(17)) # New hidden layer with 4 params
model.add(Dense(13)) # New hidden layer with 4 params
model.add(Dense(12)) # New hidden layer with 4 params
model.add(Dense(11)) # New hidden layer with 4 params
model.add(Dense(7)) # New hidden layer with 4 params
model.add(Dense(5)) # New hidden layer with 4 params
model.add(Dense(5)) # New hidden layer with 4 params
model.add(Dense(5)) # New hidden layer with 4 params

model.add(Dense(4, activation = 'softmax'))


In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(xtrain,ytrain, batch_size =100, epochs = 100)

Epoch 1/100
180/180 [==============================] - 1s 2ms/step - loss: 1.2010 - accuracy: 0.4358
Epoch 2/100
180/180 [==============================] - 0s 2ms/step - loss: 0.6473 - accuracy: 0.7652
Epoch 3/100
180/180 [==============================] - 0s 2ms/step - loss: 0.6182 - accuracy: 0.7662
Epoch 4/100
180/180 [==============================] - 0s 2ms/step - loss: 0.6216 - accuracy: 0.7672
Epoch 5/100
180/180 [==============================] - 0s 2ms/step - loss: 0.6225 - accuracy: 0.7661
Epoch 6/100
180/180 [==============================] - 0s 2ms/step - loss: 0.6259 - accuracy: 0.7623
Epoch 7/100
180/180 [==============================] - 0s 2ms/step - loss: 0.6170 - accuracy: 0.7660
Epoch 8/100
180/180 [==============================] - 0s 2ms/step - loss: 0.6237 - accuracy: 0.7624
Epoch 9/100
180/180 [==============================] - 0s 2ms/step - loss: 0.6209 - accuracy: 0.7648
Epoch 10/100
180/180 [==============================] - 0s 2ms/step - loss: 0.6156 - accura

In [ ]:
ypred = model.predict(xtest)
ytestclass = np.argmax(ytest, axis =1)
ypredclass = np.argmax(ypred, axis = 1)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(ytestclass,ypredclass))
print(confusion_matrix(ytestclass,ypredclass))

              precision    recall  f1-score   support

           0       0.79      0.99      0.88      1571
           1       0.00      0.00      0.00        12
           2       0.31      0.01      0.02       404
           3       0.00      0.00      0.00        13

    accuracy                           0.78      2000
   macro avg       0.27      0.25      0.23      2000
weighted avg       0.68      0.78      0.69      2000

[[1560    0   11    0]
 [  12    0    0    0]
 [ 399    0    5    0]
 [  13    0    0    0]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
(3166)/(3166+39+815+30)

0.7817283950617284

In [ ]:
(3116)/(3116+39+815+30)

0.779

In [ ]:
(4621)/(4621+52+1284+43)

0.7701666666666667

In [ ]:
(4589)/(4589+1233+47+58+36+35)

0.7650883627875958

In [ ]:
1565/(1560+11+5+12+399+12)

0.7828914457228614